**To run this code you must install all the dependencies in the `docker/requirements.txt` in your personal coputer**

# Cargue de Usuarios a SQL

In [27]:
import pandas as pd
import sqlalchemy

import os
import dotenv

dotenv.load_dotenv()

True

In [28]:
# Configuración de la cadena de conexión
SQL_USER=os.getenv("SQL_USER")
SQL_PASSWORD=os.getenv("SQL_PASSWORD")
SQL_HOST=os.getenv("SQL_HOST")
SQL_PORT=os.getenv("SQL_PORT")
SQL_DATABASE=os.getenv("SQL_DATABASE")

# Cadena de conexión utilizando pyodbc
connection_string = f"mssql+pyodbc://{SQL_USER}:{SQL_PASSWORD}@{SQL_HOST}:{SQL_PORT}/{SQL_DATABASE}?driver=SQL+Server"

# Crear el motor de conexión
engine = sqlalchemy.create_engine(connection_string)

In [29]:
data = pd.read_csv("./data/vendedores_projecto_final_bases_de_datos.csv")
data

,numero_vendedor,username,password,first_name,last_name,title
0,NV-0,carmas,supersecret,Carlos,Masmela,Sales Manager
1,NV-1,yupe,supersecret,Yudy,Pedraza,Sales Executive
2,NV-2,fama,supersecret,Fabian,Madero,VP Sales
3,NV-3,elsa,supersecret,Elkin,Sanchez,EVP Sales
4,NV-4,gial,supersecret,Giovanny,Albarracin,Sales Assistant


In [30]:
data.to_sql('users', con=engine, if_exists='replace', index=False)

-1

# Cargue de Predicciones

In [31]:
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
import pymongo

import sqlalchemy

import os
import dotenv

dotenv.load_dotenv()

import pandas as pd
import json

import joblib

In [56]:
def read_data_mongodb(USERNAME: str, PASSWORD: str, SERVER: str) -> json:
    """Función que lee los datos de MongoDB

    Args:
        USERNAME (str): Usuario de la base de datos
        PASSWORD (str): Contraseña de la base de datos
        SERVER (str): Servidor de la base de datos

    Returns:
        json: Datos finales
    """

    uri = f"mongodb+srv://{USERNAME}:{PASSWORD}@{SERVER}.czf3nfw.mongodb.net/?retryWrites=true&w=majority&appName={SERVER}"

    # Crear cliente y conectar al servidor
    client = MongoClient(uri, server_api=ServerApi("1"))

    # Enviar PING para conocer si hay conexión  exitosa
    try:
        client.admin.command("ping")
        print("Pinged your deployment. You successfully connected to MongoDB!")
    except Exception as e:
        print(e)

    # Conectarse a la base de datos
    db = client["final_project"]

    # Conectarse a la colección
    collection_oppty = db["opportunities"]

    # Leer datos excluyendo el campo '_id'
    documents = collection_oppty.find(
        {},
        {
            "_id": 0,
            "numero_opportunidad": 1,
            "unidad_negocio": 1,
            "etapas": 1,
            "tipos": 1,
            "acv": 1,
            "tcv": 1,
            "margen_ganancia": 1,
            "regiones": 1,
            "territorios": 1,
            "industrias": 1,
            "edad_oferta": 1,
            "lead_source": 1,
        },
    )

    # Convertir a DataFrame
    df = pd.DataFrame(list(documents))

    return df.to_json(orient="records")


def predict_win_or_loose(data) -> json:
    """Predecir si se ganará o se perderá

    Returns:
        json: Datos predicción
    """
    # Tomar la data del paso anterior
    # data = context["task_instance"].xcom_pull(task_ids="read_data_mongodb")
    df = pd.read_json(data, orient="records")

    # Leer modelo guardado
    model = joblib.load("./model/ml_model.joblib")

    # Predecir si se ganará o se perderá
    y_pred = model.predict(df.drop(columns=["etapas", "numero_opportunidad"]))

    # Transformar solución a cargar a Redis (Numero Oppty y Predicción)
    df_resul = pd.concat([df["numero_opportunidad"], pd.DataFrame(y_pred)], axis=1)
    df_resul.columns = ["numero_opportunidad", "prediccion_modelo"]

    return df_resul.to_json(orient="records")


def save_data_sql_server(
    SQL_USER: str, SQL_PASSWORD: str, SQL_HOST: str, SQL_PORT: str, SQL_DATABASE: str, data, data_inicial #, **context
) -> None:
    """
    Graba datos en redis
    """
    # Tomar la data del paso anterior
    # data = context["task_instance"].xcom_pull(task_ids="predict_win_or_loose")
    df_resul = pd.read_json(data_inicial, orient="records")
    df_prediccion = pd.read_json(data, orient="records")
    df_prediccion=df_prediccion.drop(["numero_opportunidad"], axis=1)
    df_consolidado = pd.concat([df_resul, df_prediccion], axis=1).sort_values(by="numero_opportunidad")
    # Cadena de conexión utilizando pyodbc
    connection_string = f"mssql+pyodbc://{SQL_USER}:{SQL_PASSWORD}@{SQL_HOST}:{SQL_PORT}/{SQL_DATABASE}?driver=SQL+Server"

    # Crear el motor de conexión
    engine = sqlalchemy.create_engine(connection_string)

    # Oportunidad de ventas a SQL
    df_consolidado.to_sql('oportunidades', con=engine, if_exists='replace', index=False)

    return df_consolidado

In [35]:
df_mongodb = read_data_mongodb(USERNAME=os.getenv("MONGODB_USER"), PASSWORD=os.getenv("MONGODB_PASSWORD"), SERVER=os.getenv("MONGODB_SERVER"))
df_prediccion = predict_win_or_loose(df_mongodb)

Pinged your deployment. You successfully connected to MongoDB!


In [57]:
df_consolidado = save_data_sql_server(SQL_USER= os.getenv("SQL_USER"), SQL_PASSWORD= os.getenv("SQL_PASSWORD"), SQL_HOST= os.getenv("SQL_HOST"), SQL_PORT= os.getenv("SQL_PORT"), SQL_DATABASE= os.getenv("SQL_DATABASE"), data_inicial=df_prediccion, data=df_mongodb)
df_consolidado

,numero_opportunidad,prediccion_modelo,unidad_negocio,etapas,tipos,acv,tcv,margen_ganancia,regiones,territorios,industrias,edad_oferta,lead_source
1,OP-00000,Lost,BU-05,Lost,TY-04,476458.92,943194.18,0.06,RE-02,TE-15,IN-23,173,LS-06
35,OP-00001,Won,BU-05,Won,TY-00,8715.31,8715.31,0.07,RE-02,TE-01,IN-37,0,LS-06
34,OP-00002,Lost,BU-05,Lost,TY-04,140992.94,140992.94,0.19,RE-02,TE-15,IN-10,173,LS-14
55,OP-00003,Won,BU-05,Won,TY-00,1333.11,1333.11,0.19,RE-02,TE-00,IN-55,0,LS-11
11,OP-00004,Lost,BU-05,Lost,TY-05,388946.05,388946.05,0.28,RE-02,TE-10,IN-46,531,LS-28
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13785,OP-13786,Lost,BU-07,Lost,TY-05,197076.01,197076.01,0.29,RE-03,TE-14,IN-51,124,LS-11
13790,OP-13787,Won,BU-07,Won,TY-05,227457.25,227457.25,0.27,RE-03,TE-09,IN-39,20,LS-04
13783,OP-13788,Lost,BU-07,Lost,TY-04,291709.54,388946.05,0.17,RE-03,TE-04,IN-49,156,LS-12
13766,OP-13789,Lost,BU-07,Lost,TY-04,60286.64,60286.64,0.06,RE-03,TE-12,IN-31,181,LS-28
